# Sampling

[Preparation](#Preparation)

Constants

Functions

[2018-03-23 - 2018-04-27 Online campaign - 1.52.2](#2018-03-23---2018-04-27-Online-campaign---1.52.2)

[2018-04-27 - 2018-07-05 Online campaign - 1.60](#2018-04-27---2018-07-05-Online-campaign---1.60)

[2018-04-10 - 2018-04-28 Playtest - 1.52.2 & 1.60](#2018-04-10---2018-04-28-Playtest---1.52.2-&-1.60)

  - [Dates](#Dates)
  - [Samples](#Samples)
  
[Sample analysis](#Sample-analysis)

  Checks on the profiles counts - search for duplicates  
  Global analysis of player paths

Checks on the profiles counts - search for duplicates

Global analysis of player paths

[Sankey diagram](#Sankey-diagram)

## Preparation

In [ ]:
%run "../Functions/2. Google form analysis.ipynb"

In [ ]:
#rmdf1522.columns
#'coordinates', 'customData', 'customData.biobrick', 'customData.chapter', 'customData.controls', 'customData.count', 'customData.device', 'customData.devices', 'customData.dnabit', 'customData.duration', 'customData.energy', 'customData.graphics', 'customData.language', 'customData.life', 'userId', 'customData.message', 'customData.nanobot', 'customData.newtab', 'customData.option', 'customData.plasmid', 'customData.platform', 'customData.sametab', 'customData.slot', 'customData.sound', 'customData.source', 'customData.total', 'id', 'playerCustomData', 'sessionId', 'section', 'serverTime', 'type', 'userTime'

In [ ]:
#testUserId1 = 'f4589007-0809-4673-ad50-d985abe4fa65'
#testUserId2 = '653f174e-c23f-4d3a-94f6-d26b53ac7168'
#gform[gform[QUserId] == testUserId2]
#testUserId2 in testUsers

In [ ]:
displayResults = False

# Constants

In [ ]:
# rows
phase1Label = 'phase1'
phase2Label = 'phase2'
totalLabel = 'total'

# columns
playersCountLabel = 'players'
respondentsCountLabel = 'respondents'
respondentsExploitableCountLabel = 'exploitable respondents'
respondentsTwiceCountLabel = 'twice respondents'
respondentsTwiceVolunteersCountLabel = 'volunteers'
prefixFemale = 'f '
prefixMale = 'm '

# graph titles and axes
chainingString = ", "
totalSuffix = chainingString + totalLabel
phase1Suffix = chainingString + phase1Label
phase2Suffix = chainingString + phase2Label
twiceRespondentsSuffix = chainingString + respondentsTwiceCountLabel
volunteersSuffix = chainingString + respondentsTwiceVolunteersCountLabel

ageTitleEnglish = "Age of participants"
ageVolunteersTitleEnglish = "Age of volunteers"
ageXLabelEnglish = "Age"
ageYLabelEnglish = "Count"
unknownAgeLabelEnglish = "unknown"

ageTitleFrench = "Âge des participants"
ageFemaleTitleFrench = "Âge des participantes"
ageXLabelFrench = "Âge"
ageYLabelFrench = "Nombre"
unknownAgeLabelFrench = "inconnu"

genderLabelsEnglish = ['Females', 'Males', 'Other/Unspecified/Unknown']
genderLabelsFrench = ['Femmes', 'Hommes', 'Autres/Non précisé/Inconnu']
genderTitleEnglish = 'Gender of respondents'
genderExploitableTitleEnglish = 'Gender of exploitable respondents'
genderTitleFrench = 'Sexe des personnes interrogées'

playtestStatistics = pd.DataFrame(
    data = np.nan,
    index = [phase1Label, phase2Label, totalLabel],
    columns = [playersCountLabel]
)

# Functions

In [ ]:
def getFormattedPercentageAndAbsoluteValueFunction(total):
    return lambda p: '{:1.1f}%'.format(p) + ' ({:.0f})'.format(round(p * total / 100))

In [ ]:
# problem: counts UserIds, but there can be more than one subject per UserId
def setCount(_gfDF, rowLabel, columnLabel, useProfiles=True):
    if useProfiles:
        playtestStatistics.loc[rowLabel, columnLabel] = len(getUniqueProfiles(_gfDF))
    else:
        playtestStatistics.loc[rowLabel, columnLabel] = len(_gfDF[QUserId].unique())
    return playtestStatistics.loc[rowLabel, columnLabel]

def setGenderedCount(sampledGform, rowLabel, columnLabel, plotPie = False, title = ''):
    femaleSample = sampledGform[sampledGform[QGender] == "Female"]
    maleSample = sampledGform[sampledGform[QGender] == "Male"]

    tCount = setCount(sampledGform, rowLabel, columnLabel)
    fCount = setCount(femaleSample, rowLabel, prefixFemale + columnLabel)
    mCount = setCount(maleSample, rowLabel, prefixMale + columnLabel)
    oCount = tCount-fCount-mCount
    
    if plotPie:
        plt.pie([fCount, mCount, oCount], labels=genderLabelsEnglish,\
                autopct=getFormattedPercentageAndAbsoluteValueFunction(tCount),\
                shadow=True)
        plt.title(title)
        plt.show()

In [ ]:
def getUniqueProfiles(_gfDF):
    playtestProfiles = _gfDF.loc[:,[QUserId, QAge, QGender]]
    playtestProfiles.drop_duplicates(subset=[QUserId, QAge, QGender], keep='first', inplace=True)
    return playtestProfiles

In [ ]:
# samples using UserIds associated with unique profiles
def getUserIdsWithUniqueProfile(_gfDF):
    multipleProfiles = _gfDF.loc[:, [QUserId, QAge, QGender]].drop_duplicates()[QUserId].value_counts()
    multipleProfiles = multipleProfiles[multipleProfiles != 1]    
    return _gfDF[~_gfDF[QUserId].isin(multipleProfiles.index)]

In [ ]:
#multipleProfiles = gfdfPlaytest.loc[:, [QUserId, QAge, QGender]].drop_duplicates()[QUserId].value_counts()
#multipleProfiles = multipleProfiles[multipleProfiles != 1]
#for userId in multipleProfiles.index:
#    print(str(gfdfPlaytest[gfdfPlaytest[QUserId] == userId].loc[:, [QUserId, QAge, QGender]]))

In [ ]:
def getTicksByStep(minValue, maxValue, step):
    start = minValue + step - (minValue % step)
    end = maxValue - (maxValue % step) + step

    ticks = [x for x in range(start, end, step)]
    ticks = np.append([minValue], ticks)
    ticks = np.append(ticks, [maxValue])

    return ticks

In [ ]:
def getTicks(minValue, maxValue, step = 0, labelCount = 0):
    if step == 0 and labelCount != 0:
        step = int((maxValue - minValue + 1) / labelCount)
    return getTicksByStep(minValue, maxValue, step)

In [ ]:
def plotAge(_gfDF, title = ageTitleEnglish, xlabel = ageXLabelEnglish, ylabel = ageYLabelEnglish):
    playtestProfiles = getUniqueProfiles(_gfDF)

    ages = playtestProfiles[QAge].value_counts().sort_index()
    minAge = ages.index.min()
    maxAge = ages.index.max()
    ages = pd.Series(ages, index=range(minAge, maxAge+1)).fillna(0)
    
    yticks = getTicks(int(ages.min()), int(ages.max()), labelCount = 5)
    ax = ages.plot(kind='bar', yticks=yticks)
    
    #ax = ages.plot(kind='bar')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    
    # xticks like yticks doesn't work
    # workaround
    xticks = getTicks(minAge, maxAge, step = 5)
    newTicks = xticks - minAge
    newLabels = xticks
    ax.xaxis.set_ticks(newTicks)
    ax.xaxis.set_ticklabels(newLabels)
    
    plt.title(title)
    plt.show()

In [ ]:
#_gfDF = gfdfPlaytest.copy()
#pivotalAgesList = [3,15,25,35,45,55]
#title = ageTitleEnglish

def plotAgeGroups(_gfDF, pivotalAgesList = [], title = ageTitleEnglish,\
                  showCount = False, unknownAgeLabel = unknownAgeLabelEnglish):
    playtestProfiles = getUniqueProfiles(_gfDF)

    ages = playtestProfiles[QAge]
    maxAge = ages.max()
    minAge = ages.min()
    otherAge = maxAge + 1

    pivotalAgesList.sort()
    pivotalAgesList = [x for x in pivotalAgesList if (x < maxAge and x > minAge)]
    pivotalAgesList = np.append(pivotalAgesList, [maxAge, otherAge])

    ageBins = pd.Series(0,index=pivotalAgesList)
    labels = pd.Series('+++',index=pivotalAgesList)
    previous = str(minAge)

    for pivot in pivotalAgesList:
        if pivot != otherAge:
            ageBins[pivot] = len(ages[ages <= pivot])
            labels[pivot] = previous + '-' + str(pivot)
            if showCount:
                labels[pivot] += " (" + str(ageBins[pivot]) + ")"
            ages = ages[ages > pivot]
            previous = str(pivot)
        else:
            ages = playtestProfiles[QAge]
            ageBins[otherAge] = len(ages[pd.isnull(ages)])
            labels[otherAge] = unknownAgeLabel
            if showCount:
                labels[otherAge] += " (" + str(ageBins[otherAge]) + ")"

    plt.pie(ageBins.values, labels=labels,\
            #autopct='%1.1f%%',\
            autopct=getFormattedPercentageAndAbsoluteValueFunction(len(playtestProfiles)),\
            shadow=True)
    plt.title(title)
    plt.show()

In [ ]:
def plotGender(_gfDF, title = '', labels = genderLabelsEnglish):
    playtestProfiles = getUniqueProfiles(_gfDF)

    fCount = len(playtestProfiles[playtestProfiles[QGender] == 'Female'])
    mCount = len(playtestProfiles[playtestProfiles[QGender] == 'Male'])
    tCount = len(playtestProfiles)
    oCount = tCount-fCount-mCount

    plt.pie([fCount, mCount, oCount], labels=labels,\
            autopct=getFormattedPercentageAndAbsoluteValueFunction(tCount),\
            shadow=True)
    plt.title(title)
    plt.show()
    print("t="+str(tCount))
    print("f="+str(fCount))
    print("m="+str(mCount))
    print("o="+str(oCount))

In [ ]:
def getDuplicates(_gfDF, columns):
    sampled = _gfDF.loc[:,columns]
    return sampled,sampled[sampled.duplicated()]

In [ ]:
#_gfDF = gfdfPlaytest.copy()
#columns = [QUserId, QAge, QGender, QTemporality]
def getTimedAssembledDuplicates(_gfDF, columns, displayColumns = []):
    if len(displayColumns) == 0:
        displayColumns = columns
        
    sampled,duplicates = getDuplicates(_gfDF, columns)
    dupIndex = pd.Index([])
    for dup in duplicates.values:
        dupbool = [(x == dup).all() for x in sampled.values]
        dupIndex = dupIndex.union(sampled.index[dupbool])
    result = _gfDF.loc[dupIndex, displayColumns].sort_values(by = displayColumns)
    return result

In [ ]:
#_gformDF = incorrectPairsSurveys.copy()
#_gformDF = gform.copy()

def getPerfectPretestPostestPairs(_gformDF):
    #counter = 0
    #userIdsToKeep = _gformDF[QUserId].unique()
    #print("step" + str(counter) + ": " + str(len(userIdsToKeep)))
    #counter+=1

    # remove incomplete data
    #  consequently removes posttests without pretests
    _gformDF2 = getWithoutIncompleteAnswers(_gformDF)
    #print("step" + str(counter) + ": " + str(len(_gformDF2[QUserId].unique())))
    #counter+=1

    # keep userIds with exactly 1 pretest and posttest
    #userIdsToKeep = []
    valueCounts = _gformDF2[_gformDF2[QTemporality] == answerTemporalities[0]][QUserId].value_counts()
    _gformDF3 = _gformDF2[_gformDF2[QUserId].isin(valueCounts[valueCounts == 1].index.values)]
    #print("step" + str(counter) + ": " + str(len(_gformDF3[QUserId].unique())))
    #counter+=1

    valueCounts = _gformDF3[_gformDF3[QTemporality] == answerTemporalities[1]][QUserId].value_counts()
    _gformDF4 = _gformDF3[_gformDF3[QUserId].isin(valueCounts[valueCounts == 1].index.values)]
    #print("step" + str(counter) + ": " + str(len(_gformDF4[QUserId].unique())))
    #counter+=1
    
    def getFloorDayFromUserIdAndTemporality(userId,i,df):
        return df[(df[QUserId] == userId)\
              &(df[QTemporality] == answerTemporalities[i])]\
            .iloc[0][QTimestamp].floor('d')

    # only keep those with pretest-posttest pairs on the same day
    userIds = _gformDF4[QUserId].unique()
    userIdsDF = pd.DataFrame(pd.NaT, index=userIds, columns=["0","1"])
    for i in [0, 1]:
        userIdsDF[str(i)] = pd.Series([getFloorDayFromUserIdAndTemporality(x,i,_gformDF4) for x in userIds], index = userIds)
    userIds = userIdsDF.index[userIdsDF.loc[:,"0"] == userIdsDF.loc[:,"1"]]
    _gformDF5 = _gformDF4[_gformDF4[QUserId].isin(userIds)]
    #print("step" + str(counter) + ": " + str(len(_gformDF5[QUserId].unique())))
    #counter+=1

    return _gformDF5

In [ ]:
def getVolunteers(_gfDF):
    volunteerUserIds = _gfDF[_gfDF[QVolunteer].isin(yesNoPositives)][QUserId]
    return _gfDF[_gfDF[QUserId].isin(volunteerUserIds)]

In [ ]:
def getMaxPlayDuration(rmdf, gfdf, printDebugMessages = True):
    maxMinutesCount = 1
    minuteCounts = pd.Series(index=rmdf[QUserId].unique(), data=1)
    
    _progress = IntProgress(min=0, max=len(rmdf[QUserId].unique()))
    display(_progress)
    
    for userId in rmdf[QUserId].unique():
        
        _progress.value += 1
        
        times = rmdf[rmdf[QUserId] == userId]['userTime']
        times.index = range(len(times))
        
        days = times.dt.date
        # compute day of experiment
        ratios = [len(days[days == d])/len(days) for d in days.unique()]
        ratioratios = pd.Series([r/max(ratios) for r in ratios], index = days.unique())
        importantDays = days.unique()[ratioratios > 0.05]
        if len(importantDays) > 1:
            if printDebugMessages:
                print("more than one important day!")
        
        times = times.dt.floor('min')
        times = times[times.dt.date.isin(importantDays)]
        startGraph = times.min()
        endGraph = times.max()

        if(len(gfdf) > 0):
            surveySubmissions = gfdf[gfdf[QUserId] == userId]
            surveySubmissionsTimes = surveySubmissions[QTimestamp].dt.floor('min')
            startGraph = min(startGraph, surveySubmissionsTimes.min())
            endGraph = max(endGraph, surveySubmissionsTimes.max())
        minutesCount = int((endGraph - startGraph).total_seconds()/60)
        minuteCounts[userId] = minutesCount
        if minutesCount > maxMinutesCount:
            maxMinutesCount = minutesCount
            
    _progress.close()
    del _progress
    return maxMinutesCount, minuteCounts

In [ ]:
#userId = duplicates.loc[666, QUserId]
#nPretestSubmissions = 0
#userId = "29b739fc-4f9f-4f5e-bfee-8ba12de4b7fa"
#rmdf = rmdfConcat
#surveys = gform.copy()

def plotEventsPerMinute(userId, rmdf, surveys = [], nPretestSubmissions = 0, ax = np.nan, printDebugMessages = False, minBinCount = 1):#, vmax = 60):
    if printDebugMessages:
        print("\n\nuserId="+str(userId))
    times = rmdf[rmdf[QUserId] == userId]['userTime']
    times.index = range(len(times))
    days = times.dt.date
    # compute day of experiment
    ratios = [len(days[days == d])/len(days) for d in days.unique()]
    ratioratios = pd.Series([r/max(ratios) for r in ratios], index = days.unique())
    importantDays = days.unique()[ratioratios > 0.05]
    if len(importantDays) > 1:
        if printDebugMessages:
            print("more than one important day!")
    times = times.dt.floor('min')
    times = times[times.dt.date.isin(importantDays)]
    startGraph = times.min()
    endGraph = times.max()
    if printDebugMessages:
        print("\nevents:\nfirst:" + str(startGraph) + "\nlast:" + str(endGraph))
    

    if pd.isnull(ax):
        fig = plt.figure()
        ax = fig.add_subplot(111)

    xrot = 45
    alpha = 0.5

    if(len(surveys) > 0):
        surveySubmissions = surveys[surveys[QUserId] == userId]
        surveySubmissionsTimes = surveySubmissions[QTimestamp].dt.floor('min')
        startGraph = min(startGraph, surveySubmissionsTimes.min())
        endGraph = max(endGraph, surveySubmissionsTimes.max())

        pretestSubmissions = surveySubmissions[surveySubmissions[QTemporality] == answerTemporalities[0]][QTimestamp].dt.floor('min')
        posttestSubmissions = surveySubmissions[surveySubmissions[QTemporality] == answerTemporalities[1]][QTimestamp].dt.floor('min')

        if len(pretestSubmissions) > 0:
            ax2 = ax.twinx()
            ax2.set_yticks([])
            ax2.set_yticklabels([])
        if len(posttestSubmissions) > 0:
            ax3 = ax.twinx()
            ax3.set_yticks([])
            ax3.set_yticklabels([])
       
    if len(times) == 0:
        if printDebugMessages:
            print("len(times) == 0")
    
    minutesCount = int((endGraph - startGraph).total_seconds()/60)
    binCount = max(minBinCount,minutesCount)
    endGraph = startGraph + pd.Timedelta(minutes=binCount)
    plotRange = (startGraph,endGraph)    
    
    #kwds = dict('vmax'=vmax)
    #kwds = {'vmax': vmax}
    #ymin, ymax = plt.ylim()
    #plt.ylim(0, vmax)
    times.hist(bins=binCount, ax=ax, xrot=xrot, color='blue', alpha=alpha, range=plotRange, grid=False)
    #plt.ylim(ymin, ymax)
    
    if(len(surveys) > 0):

        if len(pretestSubmissions) > 0:
            if printDebugMessages:
                print("\npretests:\n" + str(pretestSubmissions))
            pretestSubmissions[nPretestSubmissions:].hist(bins = 20, ax = ax2, xrot = xrot, color='green', alpha=alpha, range=plotRange, grid=False)

        if len(posttestSubmissions) > 0:
            if printDebugMessages:
                print("\nposttests:\n" + str(posttestSubmissions))
            posttestSubmissions.hist(bins = 20, ax = ax3, xrot = xrot, color='red', alpha=alpha, range=plotRange, grid=False)

    #plt.show()

In [ ]:
def getResetConfigurationEvents(rmdf):
    rmdf[(rmdf['type'] == 'selectmenu') & (rmdf['customData.option'] == '"RESETCONFIGURATION"')].loc[:, [QUserId, 'userTime', 'customData.option']]

In [ ]:
# returns the userIds with multiple gender-age profiles
def getProfileCounts(_gfDF, n = 0):
    sampled = _gfDF.loc[:,[QUserId, QGender, QAge]].drop_duplicates()[QUserId].value_counts().sort_values(ascending=False)
    return sampled[sampled >= n]

In [ ]:
def getRmdfFromPlatform(_rmDF, platform):
    rmDF = _rmDF[_rmDF['customData.platform'] == platform]
    rmDF = _rmDF[_rmDF[QUserId].isin(rmDF[QUserId])]
    return rmDF

In [ ]:
def getGfdfFromRmdf(_gfDF, _rmDF, useDate = True):
    if useDate:
        dates = [_rmDF['userTime'].min(),_rmDF['userTime'].max()]
        _gfDF = _gfDF[(_gfDF[QTimestamp] >= dates[0]) & (_gfDF[QTimestamp] <= dates[1])]
    _gfDF = getWithoutTest(_gfDF[_gfDF[QUserId].isin(_rmDF[QUserId].unique())])
    return _gfDF

In [ ]:
def getRmdfFromGfdf(rmdf, gfdf, clamp = False):
    uniqueUserIds = gfdf[QUserId].unique()
    result = rmdf[rmdf[QUserId].isin(uniqueUserIds)]
    if clamp:
        
        result2 = pd.DataFrame()
        
        _progress = IntProgress(min=0, max=len(uniqueUserIds))
        display(_progress)
        
        for userId in uniqueUserIds:
            _progress.value += 1
            
            surveys = gfdf[gfdf[QUserId] == userId]
            pretest = surveys[surveys[QTemporality]==answerTemporalities[0]].iloc[0]
            posttest = surveys[surveys[QTemporality]==answerTemporalities[1]].iloc[0]
            
            filtered = result[(result[QUserId]==userId)\
                   & (pretest[QTimestamp] < result['userTime'])\
                   & (posttest[QTimestamp] > result['userTime'])]
            
            if len(result2) > 0:
                result2 = pd.concat([result2, filtered])
            else:
                result2 = filtered
        result = result2
        _progress.close()
        del _progress
    return result

In [ ]:
def getSamples(gfdf, rmdf, platform, clamp = True, isInTimeSlot = np.nan):
        
    rmdfPlatform = getRmdfFromPlatform(rmdf, platform)
    gfdfPlatform = getGfdfFromRmdf(gfdf, rmdfPlatform)
    
    if pd.isnull(isInTimeSlot):
        gfdfTimed = gfdfPlatform
        rmdfTimed = rmdfPlatform
    else:
        gfdfTimed = gfdfPlatform[gfdfPlatform[QTimestamp].apply(isInTimeSlot)]
        rmdfTimed = rmdfPlatform[rmdfPlatform[QUserId].isin(gfdfTimed[QUserId])]
        rmdfTimed = rmdfTimed[rmdfTimed['userTime'].apply(isInTimeSlot)]
    
    # sample by filtering out userIds with multiple profiles
    #  better solution: take those multiple profiles into account, eg by creating new userIds for them
    gfdfUniqueProfiles = getUserIdsWithUniqueProfile(gfdfTimed)
    # sample by keeping only pretest-posttest pairs
    gfdfPretestPosttestUniqueProfiles = getPerfectPretestPostestPairs(gfdfUniqueProfiles)
    # sample according to the answer to the Volunteering question
    gfdfPretestPosttestUniqueProfilesVolunteers = getVolunteers(gfdfPretestPosttestUniqueProfiles)
    
    rmdfUniqueProfiles = getRmdfFromGfdf(rmdfTimed, gfdfUniqueProfiles)
    rmdfPretestPosttestUniqueProfiles = getRmdfFromGfdf(rmdfUniqueProfiles, gfdfPretestPosttestUniqueProfiles, clamp = clamp)
    rmdfPretestPosttestUniqueProfilesVolunteers = getRmdfFromGfdf(rmdfPretestPosttestUniqueProfiles,\
                                                                  gfdfPretestPosttestUniqueProfilesVolunteers)
        
    return rmdfPlatform, rmdfTimed, rmdfUniqueProfiles, rmdfPretestPosttestUniqueProfiles, rmdfPretestPosttestUniqueProfilesVolunteers,\
gfdfPlatform, gfdfTimed, gfdfUniqueProfiles, gfdfPretestPosttestUniqueProfiles, gfdfPretestPosttestUniqueProfilesVolunteers

In [ ]:
def printAllSampleCounts(
    baseGfdf,
    gfdfPlatform,
    gfdfTimeConstrained,
    gfdfPlatformUniqueProfiles,
    gfdfPlatformPretestPosttestUniqueProfiles,
    gfdfPlatformPretestPosttestUniqueProfilesVolunteers,
    title="",
):
    if displayResults:
        pretests = gfdfTimeConstrained[gfdfTimeConstrained[QTemporality] == answerTemporalities[0]][QUserId]
        posttests = gfdfTimeConstrained[gfdfTimeConstrained[QTemporality] == answerTemporalities[1]][QUserId]
        both = pd.concat([pretests[pretests.isin(posttests)], posttests[posttests.isin(pretests)]])

        pretestsUnique = set(pretests)
        posttestsUnique = set(posttests)    
        bothUnique = pretestsUnique.intersection(posttestsUnique)

        print(title)
        print("Base="+str(len(baseGfdf)))
        print("Platform="+str(len(gfdfPlatform)))
        print("TimeConstrained="+str(len(gfdfTimeConstrained)))
        print("pretests="+str(len(pretests)))
        print("posttests="+str(len(posttests)))
        print("both="+str(len(both)))
        print("UniqueProfiles="+str(len(gfdfPlatformUniqueProfiles)))
        print("PretestPosttestUniqueProfiles="+str(len(gfdfPlatformPretestPosttestUniqueProfiles)))
        print("PretestPosttestUniqueProfilesVolunteers="+str(len(gfdfPlatformPretestPosttestUniqueProfilesVolunteers)))

        print()
        print("unique userIds Base="+str(baseGfdf[QUserId].nunique()))
        print("unique userIds Platform="+str(gfdfPlatform[QUserId].nunique()))
        print("unique userIds TimeConstrained="+str(gfdfTimeConstrained[QUserId].nunique()))
        print("unique userIds pretests="+str(len(pretestsUnique)))
        print("unique userIds posttests="+str(len(posttestsUnique)))
        print("unique userIds both="+str(len(bothUnique)))
        print("unique userIds UniqueProfiles="+str(gfdfPlatformUniqueProfiles[QUserId].nunique()))
        print("unique userIds PretestPosttestUniqueProfiles="+str(gfdfPlatformPretestPosttestUniqueProfiles[QUserId].nunique()))
        print("unique userIds PretestPosttestUniqueProfilesVolunteers="+str(gfdfPlatformPretestPosttestUniqueProfilesVolunteers[QUserId].nunique()))

In [ ]:
def plotUserActivity(_gfDF, _rmDF, useOneGraph = True):
    
    sizePerRow = 6
    graphsPerRow = 5
    
    i = 0
    uniques = _gfDF[QUserId].unique()
    _progress = IntProgress(min=0, max=len(uniques))
    display(_progress)
    
    if (len(uniques) > graphsPerRow) and useOneGraph:
        f, axarr = plt.subplots(len(uniques)//graphsPerRow+1, graphsPerRow, figsize=(20, (len(uniques)//graphsPerRow)*sizePerRow))
        
    for userId in uniques:
        if (len(uniques) <= graphsPerRow) or (not useOneGraph):
            if i%graphsPerRow == 0:
                f, axarr = plt.subplots(1, graphsPerRow, figsize=(20, sizePerRow))
            ax = axarr[i%graphsPerRow]
        else:
            ax = axarr[i//graphsPerRow, i%graphsPerRow] 
            
        # minBinCount is the minimal number of minutes on each subgraph
        plotEventsPerMinute(userId, _rmDF, _gfDF, ax=ax, minBinCount=30, printDebugMessages=False)#, vmax = 60)
        i += 1
        _progress.value += 1
        
    _progress.close()
    del _progress

In [ ]:
def getScoresSimplified(_gfDF):
    binarized = getAllBinarized(_gfDF)
    binarized.index = _gfDF.index
    scores = np.dot(binarized,np.ones(len(binarized.columns)))
    return scores, binarized

# Common treatment before sampling

## Common treatment

In [ ]:
sampleWhole = getWithoutTest(gform)

## Common online treatment

# 2018-03-23 - 2018-04-27 Online campaign - 1.52.2

In [ ]:
rmdfWebgl1522, rmdfWebgl1522Timed, rmdfWebgl1522UniqueProfiles, rmdfWebgl1522PretestPosttestUniqueProfiles, rmdfWebgl1522PretestPosttestUniqueProfilesVolunteers,\
gfdfWebgl1522, gfdfWebgl1522Timed, gfdfWebgl1522UniqueProfiles, gfdfWebgl1522PretestPosttestUniqueProfiles, gfdfWebgl1522PretestPosttestUniqueProfilesVolunteers\
= getSamples(sampleWhole, rmdf1522, '"webglplayer"')

In [ ]:
printAllSampleCounts(
    sampleWhole,
    gfdfWebgl1522,
    gfdfWebgl1522Timed,
    gfdfWebgl1522UniqueProfiles,
    gfdfWebgl1522PretestPosttestUniqueProfiles,
    gfdfWebgl1522PretestPosttestUniqueProfilesVolunteers,
    title="",
)

# 2018-04-27 - 2018-07-05 Online campaign - 1.60

In [ ]:
rmdfWebgl160, rmdfWebgl160Timed, rmdfWebgl160UniqueProfiles, rmdfWebgl160PretestPosttestUniqueProfiles, rmdfWebgl160PretestPosttestUniqueProfilesVolunteers,\
gfdfWebgl160, gfdfWebgl160Timed, gfdfWebgl160UniqueProfiles, gfdfWebgl160PretestPosttestUniqueProfiles, gfdfWebgl160PretestPosttestUniqueProfilesVolunteers\
= getSamples(sampleWhole, rmdf160, '"webglplayer"')

In [ ]:
printAllSampleCounts(
    sampleWhole,
    gfdfWebgl160,
    gfdfWebgl160Timed,
    gfdfWebgl160UniqueProfiles,
    gfdfWebgl160PretestPosttestUniqueProfiles,
    gfdfWebgl160PretestPosttestUniqueProfilesVolunteers,
    title="",
)

# 2018-04-10 - 2018-04-28 Playtest - 1.52.2 & 1.60

## Dates

In [ ]:
playtest2018Phase1Days = [
    pd.Timestamp('2018-04-10', tz='utc'),	# Tuesday
    pd.Timestamp('2018-04-11', tz='utc'),	# Wednesday

    pd.Timestamp('2018-04-13', tz='utc'),	# Friday
    pd.Timestamp('2018-04-14', tz='utc'),	# Saturday


    pd.Timestamp('2018-04-17', tz='utc'),	# Tuesday
    pd.Timestamp('2018-04-18', tz='utc'),	# Wednesday

    pd.Timestamp('2018-04-20', tz='utc'),	# Friday
    pd.Timestamp('2018-04-21', tz='utc'),	# Saturday


    pd.Timestamp('2018-04-24', tz='utc'),	# Tuesday
    pd.Timestamp('2018-04-25', tz='utc'),	# Wednesday
]

playtest2018Phase2Days = [
    pd.Timestamp('2018-04-27', tz='utc'),	# Friday
    pd.Timestamp('2018-04-28', tz='utc'),	# Saturday
]

playtest2018Days = np.append(playtest2018Phase1Days, playtest2018Phase2Days).tolist()

In [ ]:
playtest2018EuBerTimeStartTime = datetime.time(11, 0)
playtest2018EuBerTimeEndTime = datetime.time(19, 0)

### aliases for experiment boundaries

In [ ]:
playtest2018Phase1StartDate = pd.Timestamp.combine(min(playtest2018Phase1Days).date(), playtest2018EuBerTimeStartTime).tz_localize('Europe/Berlin')
playtest2018Phase1EndDate   = pd.Timestamp.combine(max(playtest2018Phase1Days).date(), playtest2018EuBerTimeEndTime).tz_localize('Europe/Berlin')
playtest2018Phase2StartDate = pd.Timestamp.combine(min(playtest2018Phase2Days).date(), playtest2018EuBerTimeStartTime).tz_localize('Europe/Berlin')
playtest2018Phase2EndDate   = pd.Timestamp.combine(max(playtest2018Phase2Days).date(), playtest2018EuBerTimeEndTime).tz_localize('Europe/Berlin')
playtest2018Phase1StartDate,playtest2018Phase1EndDate,playtest2018Phase2StartDate,playtest2018Phase2EndDate

In [ ]:
# warning: timestamp = pd.Timestamp('2018-04-24 11:00:11.381000+0000', tz='Europe/Berlin')
#  actually yields Timestamp('2018-04-24 13:00:11.381000+0200', tz='Europe/Berlin') because
#  '+....' codes for timezone, eg here '+0000' codes for UTC timezone

def isPlaytest(timestamp, days = playtest2018Days):
    euBerTimestamp = timestamp.tz_convert('Europe/Berlin')
    euBerTimestampTime = euBerTimestamp.time()
    result = (timestamp.tz_convert('utc').floor('d') in days)\
    and (euBerTimestampTime >= playtest2018EuBerTimeStartTime)\
    and (playtest2018EuBerTimeEndTime >= euBerTimestampTime)
    return result

def isPlaytestPhase1(timestamp):
    return isPlaytest(timestamp, days = playtest2018Phase1Days)

def isPlaytestPhase2(timestamp):
    return isPlaytest(timestamp, days = playtest2018Phase2Days)

#### isPlaytest day and hour test

In [ ]:
assert(~any([isPlaytest(x) for x in playtest2018Phase1Days]))

In [ ]:
# handpicked error cases for April 2018 Playtest
pretestPosttestPairsErrorCases = [
  # 2 posttests
    "4b19e51b-c81f-4cbb-af72-0dced3bbb52b",
    "1e6cb7d9-56f5-48a5-9bb8-14885325add6",
    "9562d4e4-3d36-4399-a95f-7784328c2683",

  # 1 posttest
    "e79d806e-3d0b-49f8-a5e8-23de5548b770",
    "8d1f1ec8-379e-4ec6-88ba-2fdc1d0989be",

  # 1 pretest
    "3006900c-37d2-44dd-a8e6-674f6fd1cae3",
    "721cb935-f939-40ae-b56f-01d97ea23604",

  # 2 posttests, 1 pretest
    "38ef3b8d-dedc-4224-be1c-5d7372973cc0",
    "96470842-aa9a-4802-9e23-38060a1980b4",

  # 2 pretests
    "f8920e82-1cdf-43a0-aedb-06ddaf9960d6",
    "86c71536-ee24-4c3d-b856-6a97b63f6217",

  # 1 pretest, 1 posttest on 2 different days
    "4cdb6030-4547-4c8f-9d9e-286cff786157",
    "9d09b624-c719-446b-a738-e7059dd1a8f4",
]
#incorrectPairsSurveys = gfdfPlaytestUniqueProfiles[gfdfPlaytestUniqueProfiles[QUserId].isin(pretestPosttestPairsErrorCases)]
#len(incorrectPairsSurveys)

In [ ]:
testCorrectTimes = [
                pd.Timestamp('11:00:00', tz='Europe/Berlin').tz_convert('utc'),
                pd.Timestamp('12:43:00', tz='Europe/Berlin').tz_convert('utc'),
                pd.Timestamp('14:58:00', tz='Europe/Berlin').tz_convert('utc'),
                pd.Timestamp('16:38:00', tz='Europe/Berlin').tz_convert('utc'),
                pd.Timestamp('19:00:00', tz='Europe/Berlin').tz_convert('utc'),
]
testIncorrectTimes = [
                pd.Timestamp('19:01:00', tz='Europe/Berlin').tz_convert('utc'),
                pd.Timestamp('21:43:00', tz='Europe/Berlin').tz_convert('utc'),
                pd.Timestamp('23:13:00', tz='Europe/Berlin').tz_convert('utc'),
                pd.Timestamp('06:41:00', tz='Europe/Berlin').tz_convert('utc'),
                pd.Timestamp('10:59:00', tz='Europe/Berlin').tz_convert('utc'),
]

for time in testCorrectTimes:
    correctPlaytestTimes = [pd.Timestamp.combine(x.date(), time.time()).tz_localize('utc') for x in playtest2018Phase1Days]
    assert(all([isPlaytest(x) for x in correctPlaytestTimes]))
    
for time in testIncorrectTimes:
    correctPlaytestTimes = [pd.Timestamp.combine(x.date(), time.time()).tz_localize('utc') for x in playtest2018Phase1Days]
    assert(~any([isPlaytest(x) for x in correctPlaytestTimes]))

## Samples
samples based on dates and platform

In [ ]:
rmdfLinux, rmdfPlaytest, rmdfPlaytestUniqueProfiles, rmdfPlaytestPretestPosttestUniqueProfiles, rmdfPlaytestPretestPosttestUniqueProfilesVolunteers,\
gfdfLinux, gfdfPlaytest, gfdfPlaytestUniqueProfiles, gfdfPlaytestPretestPosttestUniqueProfiles, gfdfPlaytestPretestPosttestUniqueProfilesVolunteers\
= getSamples(sampleWhole, rmdfConcat, '"linuxplayer"', isInTimeSlot = isPlaytest)

In [ ]:
rmdfLinux, rmdfPlaytestPhase1, rmdfPlaytestUniqueProfilesPhase1, rmdfPlaytestPretestPosttestUniqueProfilesPhase1, rmdfPlaytestPretestPosttestUniqueProfilesVolunteersPhase1,\
gfdfLinux, gfdfPlaytestPhase1, gfdfPlaytestUniqueProfilesPhase1, gfdfPlaytestPretestPosttestUniqueProfilesPhase1, gfdfPlaytestPretestPosttestUniqueProfilesVolunteersPhase1\
= getSamples(sampleWhole, rmdfConcat, '"linuxplayer"', isInTimeSlot = isPlaytestPhase1)

In [ ]:
rmdfLinux, rmdfPlaytestPhase2, rmdfPlaytestUniqueProfilesPhase2, rmdfPlaytestPretestPosttestUniqueProfilesPhase2, rmdfPlaytestPretestPosttestUniqueProfilesVolunteersPhase2,\
gfdfLinux, gfdfPlaytestPhase2, gfdfPlaytestUniqueProfilesPhase2, gfdfPlaytestPretestPosttestUniqueProfilesPhase2, gfdfPlaytestPretestPosttestUniqueProfilesVolunteersPhase2\
= getSamples(sampleWhole, rmdfConcat, '"linuxplayer"', isInTimeSlot = isPlaytestPhase2)

In [ ]:
len(gfdfPlaytestUniqueProfiles[QUserId].unique()),\
len(gfdfPlaytestUniqueProfilesPhase1[QUserId].unique()),\
len(gfdfPlaytestUniqueProfilesPhase2[QUserId].unique())

In [ ]:
printAllSampleCounts(
    sampleWhole,
    gfdfLinux,
    gfdfPlaytest,
    gfdfPlaytestUniqueProfiles,
    gfdfPlaytestPretestPosttestUniqueProfiles,
    gfdfPlaytestPretestPosttestUniqueProfilesVolunteers,
    title="Playtest total",
)

In [ ]:
printAllSampleCounts(
    sampleWhole,
    gfdfLinux,
    gfdfPlaytestPhase1,
    gfdfPlaytestUniqueProfilesPhase1,
    gfdfPlaytestPretestPosttestUniqueProfilesPhase1,
    gfdfPlaytestPretestPosttestUniqueProfilesVolunteersPhase1,
    title="Playtest Phase1",
)

In [ ]:
printAllSampleCounts(
    sampleWhole,
    gfdfLinux,
    gfdfPlaytestPhase2,
    gfdfPlaytestUniqueProfilesPhase2,
    gfdfPlaytestPretestPosttestUniqueProfilesPhase2,
    gfdfPlaytestPretestPosttestUniqueProfilesVolunteersPhase2,
    title="Playtest Phase2",
)

## Phase 1 vs Phase 2 comparison

### Scores

In [ ]:
scoresPhase1, binarizedPhase1 = getScoresSimplified(gfdfPlaytestPretestPosttestUniqueProfilesVolunteersPhase1)
scoresPhase2, binarizedPhase2 = getScoresSimplified(gfdfPlaytestPretestPosttestUniqueProfilesVolunteersPhase2)

In [ ]:
ttest = ttest_ind(scoresPhase1, scoresPhase2)
ttest
#print("t test: statistic=" + repr(ttest.statistic) + " pvalue=" + repr(ttest.pvalue))

In [ ]:
scoresPhase1, binarizedPhase1 = getScoresSimplified(gfdfPlaytestPretestPosttestUniqueProfilesPhase1)
scoresPhase2, binarizedPhase2 = getScoresSimplified(gfdfPlaytestPretestPosttestUniqueProfilesPhase2)

In [ ]:
ttest = ttest_ind(scoresPhase1, scoresPhase2)
ttest
#print("t test: statistic=" + repr(ttest.statistic) + " pvalue=" + repr(ttest.pvalue))

In [ ]:
scoresPhase1, binarizedPhase1 = getScoresSimplified(gfdfPlaytestUniqueProfilesPhase1)
scoresPhase2, binarizedPhase2 = getScoresSimplified(gfdfPlaytestUniqueProfilesPhase2)

In [ ]:
ttest = ttest_ind(scoresPhase1, scoresPhase2)
ttest
#print("t test: statistic=" + repr(ttest.statistic) + " pvalue=" + repr(ttest.pvalue))

# Sample analysis

In [ ]:
#gfdf = gfdfWebgl1522
#rmdf = rmdfWebgl1522

gfdf = gfdfWebgl160
rmdf = rmdfWebgl160

#gfdf = gfdfPlaytest
#rmdf = rmdfLinux

#gfdf = gfdfPlaytestPhase1
#rmdf = rmdfLinux

#gfdf = gfdfPlaytestPhase2
#rmdf = rmdfLinux

## Checks on the profiles counts - search for duplicates

In [ ]:
if False:
    # duplicate profiles
    #  may be people who filled in the same survey twice in a row
    #  or different people who happened to belong to same demographic group and play on same unreset computer
    #  Only way to differentiate: look at RedMetrics events
    columns = [QUserId, QAge, QGender, QTemporality]
    displayColumns = [QUserId, QAge, QGender, QTemporality, QTimestamp]
    duplicates = getTimedAssembledDuplicates(gfdf, columns, displayColumns)
    grouped = duplicates.groupby(columns)
    grouped.size()

In [ ]:
if False:
    # duplicates per day
    duplicatesPerDay = duplicates.copy()
    duplicatesPerDay.loc[:, QTimestamp] = duplicatesPerDay.loc[:, QTimestamp].apply(pd.Timestamp.date)
    duplicatesPerDay.groupby(QTimestamp).size()

In [ ]:
if False:
    # duplicates or just unpaired surveys? (ie pretest wihtout posttest or conversely)
    #  timely repartition of redmetrics events is inconclusive: analysis of RedMetrics events is necessary
    duplicates['diff'] = duplicates[QTimestamp].diff()
    duplicates

In [ ]:
if False:
    plotEventsPerMinute("e3fc1686-6802-4e36-9877-d1bb226a6f72", rmdf, gfdf)

In [ ]:
if False:
    gform[gform[QUserId] == "29b739fc-4f9f-4f5e-bfee-8ba12de4b7fa"].loc[:, [QUserId, QAge, QGender, QTemporality, QTimestamp]]

In [ ]:
if False:
    for userId in duplicates[QUserId].unique():
        print(str(gfdf[gfdf[QUserId] == userId].loc[:,displayColumns]))
        plotEventsPerMinute(userId, rmdf, gform)

In [ ]:
if False:
    getTimedAssembledDuplicates(gfdf, [QUserId, QAge, QGender], [QUserId, QAge, QGender, QTemporality, QTimestamp])

------------

## Global analysis of player paths

In [ ]:
#userIds = _gformDF4[QUserId].unique()
#userIds = pd.DataFrame(pd.NaT, index=userIds, columns=["0","1"])
#for i in [0, 1]:
#    userIds[str(i)] = pd.Series([getFloorDayFromUserIdAndTemporality(x,i) for x in userIds.index.values], index = userIds.index)
#userIds.index[userIds.loc[:,"0"] == userIds.loc[:,"1"]]

In [ ]:
if displayResults:
    maxMinutesCount, minuteCounts = getMaxPlayDuration(rmdf, gfdf, printDebugMessages = False)
    minuteCounts.name = "time spent on the game (s)"
    minuteCountsUpperBound = minuteCounts.mean() + 2 * minuteCounts.std()
    minuteCountsNoOutlier = minuteCounts.drop(index=minuteCounts[minuteCounts >= minuteCountsUpperBound].index)
    ax = sns.distplot(minuteCountsNoOutlier.sort_values(),rug=True,)
    minuteCounts.max(), minuteCounts.min(), minuteCounts.mean(), minuteCounts.median(),\
    minuteCountsNoOutlier.max(), minuteCountsNoOutlier.min(), minuteCountsNoOutlier.mean(), minuteCountsNoOutlier.median(),

In [ ]:
profilesCount = []
if False:
    profilesCount = getProfileCounts(gfdf, 1)
profilesCount

In [ ]:
if False:
    i = 0
    uniques = gfdf[QUserId].unique()#[:10]
    _progress = IntProgress(min=0, max=len(uniques))
    display(_progress)
    f, axarr = plt.subplots(len(uniques)//5+1, 5, figsize=(20,70))
    
    for userId in uniques:
        ax = axarr[i//5, i%5]
        plotEventsPerMinute(userId, rmdf, gfdf, ax=ax, minBinCount=minuteCountsNoOutlier.max())
        i += 1
        _progress.value += 1
        
    _progress.close()
    del _progress

In [ ]:
#rmdfClamped = getRmdfFromGfdf(rmdfPlaytestPretestPosttestUniqueProfilesPhase1, gfdfPlaytestPretestPosttestUniqueProfilesVolunteersPhase1, clamp = True)

In [ ]:
if displayResults:
    plotUserActivity(gfdfPlaytestPretestPosttestUniqueProfilesVolunteersPhase1,\
                rmdfPlaytestPretestPosttestUniqueProfilesVolunteersPhase1)

In [ ]:
if displayResults:
    plotUserActivity(gfdfWebgl1522,\
                rmdfWebgl1522)

In [ ]:
if displayResults:
    plotUserActivity(gfdfWebgl160,\
                rmdfWebgl160)

------------

In [ ]:
gformProfiles = getUniqueProfiles(gform)
wholeProfiles = getUniqueProfiles(sampleWhole)
linuxProfiles = getUniqueProfiles(gfdfLinux)
len(gformProfiles), len(wholeProfiles), len(linuxProfiles)

In [ ]:
playtestProfiles = getUniqueProfiles(gfdfPlaytest)
playtestProfilesPhase1 = getUniqueProfiles(gfdfPlaytestPhase1)
playtestProfilesPhase2 = getUniqueProfiles(gfdfPlaytestPhase2)

assert(len(playtestProfiles) == len(playtestProfilesPhase1) + len(playtestProfilesPhase2))

len(playtestProfiles),\
len(playtestProfilesPhase1),\
len(playtestProfilesPhase2)

In [ ]:
shouldBeEmpty = gfdfPlaytest[((gfdfPlaytest[QTimestamp] > playtest2018Phase1EndDate)&(gfdfPlaytest[QTimestamp] < playtest2018Phase2StartDate))]
assert(len(shouldBeEmpty) == 0)

In [ ]:
playtestProfiles = playtestProfiles.dropna(how='any')
playtestProfilesPhase1 = playtestProfilesPhase1.dropna(how='any')
playtestProfilesPhase2 = playtestProfilesPhase2.dropna(how='any')

assert(len(playtestProfiles) == len(playtestProfilesPhase1) + len(playtestProfilesPhase2))

len(playtestProfiles),\
len(playtestProfilesPhase1),\
len(playtestProfilesPhase2)

In [ ]:
# intersections based on userId and/or questions
s1 = pd.merge(playtestProfilesPhase1, playtestProfilesPhase2, how='inner', on=[QUserId])
s2 = pd.merge(playtestProfilesPhase1, playtestProfilesPhase2, how='inner', on=[QUserId, QAge, QGender])
len(s1), len(s2)

In [ ]:
playtestProfiles[QUserId].nunique(), playtestProfilesPhase1[QUserId].nunique(), playtestProfilesPhase2[QUserId].nunique(), 

In [ ]:
assert(rmdfPlaytest[QUserId].nunique()       == gfdfPlaytest[QUserId].nunique())
assert(rmdfPlaytestPhase1[QUserId].nunique() == gfdfPlaytestPhase1[QUserId].nunique())
assert(rmdfPlaytestPhase2[QUserId].nunique() == gfdfPlaytestPhase2[QUserId].nunique())

## Players

In [ ]:
thresholdGameEventsCount = 10
thresholdTimedelta = pd.Timedelta('0 days 00:05:00.000000')

playedEnoughTime = []
playedEnoughEvents = []

for userId in gfdfPlaytest[QUserId].unique():
    rows = rmdfPlaytest[rmdfPlaytest[QUserId] == userId]
    if len(rows) > thresholdGameEventsCount:
        playedEnoughEvents.append(userId)
    delta = rows['userTime'].max() - rows['userTime'].min()
    if delta > thresholdTimedelta:
        playedEnoughTime.append(userId)
        
len(gfdfPlaytest[QUserId].unique()), len(playedEnoughTime), len(playedEnoughEvents)

In [ ]:
setCount(gfdfPlaytestPhase1, phase1Label, playersCountLabel)
setCount(gfdfPlaytestPhase2, phase2Label, playersCountLabel)
setCount(gfdfPlaytest,       totalLabel,  playersCountLabel)
    
#assert(playtestStatistics.loc[totalLabel, playersCountLabel] == \
#    playtestStatistics.loc[phase1Label, playersCountLabel]\
#    + playtestStatistics.loc[phase2Label, playersCountLabel])

playtestStatistics

## All playtest respondents

In [ ]:
if displayResults:
    plotGender(gfdfPlaytest, genderTitleFrench)

In [ ]:
if displayResults:
    plotGender(gfdfPlaytestPhase1, genderTitleEnglish + phase1Suffix, labels = genderLabelsEnglish)

In [ ]:
if displayResults:
    plotGender(gfdfPlaytestPhase2, genderTitleEnglish + phase2Suffix)

In [ ]:
if displayResults:
    plotAge(gfdfPlaytest, ageTitleEnglish + totalSuffix)
    plotAge(gfdfPlaytestPhase1, ageTitleEnglish, xlabel = ageXLabelEnglish, ylabel = ageYLabelEnglish)
    plotAge(gfdfPlaytestPhase2, ageTitleEnglish + phase2Suffix)
    plotAge(gfdfPlaytestPretestPosttestUniqueProfilesVolunteersPhase1, ageVolunteersTitleEnglish, xlabel = ageXLabelEnglish, ylabel = ageYLabelEnglish)

In [ ]:
# For DITOS

if False:
    if displayResults:
        femaleParticipants = gfdfPlaytest[gfdfPlaytest[QGender]=="Female"]
        plotAge(femaleParticipants, ageFemaleTitleFrench)
        femaleParticipants[QAge].value_counts()
        plotAgeGroups(femaleParticipants, pivotalAgesList = [1,15,25,35,45], title = ageFemaleTitleFrench, showCount = False, unknownAgeLabel = unknownAgeLabelFrench)

In [ ]:
if displayResults:
    plotAgeGroups(gfdfPlaytest,       pivotalAgesList = [1,15,25,35,45], title = ageTitleEnglish+totalSuffix, showCount = False, unknownAgeLabel =unknownAgeLabelEnglish)
    plotAgeGroups(gfdfPlaytestPhase1, pivotalAgesList = [1,15,25,35,45], title = ageTitleEnglish+phase1Suffix, showCount = False)
    plotAgeGroups(gfdfPlaytestPhase2, pivotalAgesList = [1,15,25,35,45], title = ageTitleEnglish+phase2Suffix, showCount = False)
    plotAgeGroups(gfdfPlaytestPretestPosttestUniqueProfilesVolunteersPhase1, pivotalAgesList = [1,15,25,35,45], title = ageVolunteersTitleEnglish, showCount = False)

In [ ]:
setGenderedCount(gfdfPlaytestPhase1, phase1Label, respondentsCountLabel)
setGenderedCount(gfdfPlaytestPhase2, phase2Label, respondentsCountLabel)
setGenderedCount(gfdfPlaytest,       totalLabel,  respondentsCountLabel,\
                 plotPie = displayResults, title = genderTitleEnglish)

assert(playtestStatistics.loc[totalLabel, respondentsCountLabel] ==\
       playtestStatistics.loc[phase1Label, respondentsCountLabel]\
      + playtestStatistics.loc[phase2Label, respondentsCountLabel])

playtestStatistics

In [ ]:
setGenderedCount(gfdfPlaytestUniqueProfilesPhase1, phase1Label, respondentsExploitableCountLabel)
setGenderedCount(gfdfPlaytestUniqueProfilesPhase2, phase2Label, respondentsExploitableCountLabel)
setGenderedCount(gfdfPlaytestUniqueProfiles,       totalLabel,  respondentsExploitableCountLabel,\
                 plotPie = displayResults, title = genderExploitableTitleEnglish)

assert(playtestStatistics.loc[totalLabel, respondentsExploitableCountLabel] ==\
       playtestStatistics.loc[phase1Label, respondentsExploitableCountLabel]\
      + playtestStatistics.loc[phase2Label, respondentsExploitableCountLabel])

playtestStatistics

In [ ]:
# answers per userId
# => gform.indexes per userId
#indicesPerUserId = gform[QUserId].value_counts()
#indicesPerUserId

### Twice respondents

In [ ]:
setGenderedCount(gfdfPlaytestPretestPosttestUniqueProfilesPhase1, phase1Label, respondentsTwiceCountLabel)
setGenderedCount(gfdfPlaytestPretestPosttestUniqueProfilesPhase2, phase2Label, respondentsTwiceCountLabel)
setGenderedCount(gfdfPlaytestPretestPosttestUniqueProfiles,       totalLabel,  respondentsTwiceCountLabel,\
                 plotPie = displayResults, title = genderTitleEnglish + twiceRespondentsSuffix)

assert(playtestStatistics.loc[totalLabel, respondentsTwiceCountLabel] ==\
       playtestStatistics.loc[phase1Label, respondentsTwiceCountLabel]\
      + playtestStatistics.loc[phase2Label, respondentsTwiceCountLabel])

playtestStatistics

In [ ]:
if displayResults:
    (gfdfPlaytestPretestPosttestUniqueProfiles, 'Age repartition of pretest-posttest participants')

### Volunteers

In [ ]:
setGenderedCount(gfdfPlaytestPretestPosttestUniqueProfilesVolunteersPhase1, phase1Label, respondentsTwiceVolunteersCountLabel,\
                 plotPie = displayResults, title = genderTitleEnglish + phase1Suffix + volunteersSuffix)
setGenderedCount(gfdfPlaytestPretestPosttestUniqueProfilesVolunteersPhase2, phase2Label, respondentsTwiceVolunteersCountLabel)
setGenderedCount(gfdfPlaytestPretestPosttestUniqueProfilesVolunteers,       totalLabel,  respondentsTwiceVolunteersCountLabel,\
                 plotPie = displayResults, title = genderTitleEnglish + totalSuffix + volunteersSuffix)

assert(playtestStatistics.loc[totalLabel, respondentsTwiceVolunteersCountLabel] ==\
       playtestStatistics.loc[phase1Label, respondentsTwiceVolunteersCountLabel]\
      + playtestStatistics.loc[phase2Label, respondentsTwiceVolunteersCountLabel])

playtestStatistics

# Sankey diagram

In [ ]:
from matplotlib.sankey import Sankey

In [ ]:
groupLabel = totalLabel
players = playtestStatistics.loc[groupLabel, playersCountLabel]
minus = pd.Series()

minus[respondentsCountLabel]                = playtestStatistics.loc[groupLabel, respondentsCountLabel]\
                                                - playtestStatistics.loc[groupLabel, playersCountLabel]
minus[respondentsExploitableCountLabel]     = playtestStatistics.loc[groupLabel, respondentsExploitableCountLabel]\
                                                - playtestStatistics.loc[groupLabel, respondentsCountLabel]
minus[respondentsTwiceCountLabel]           = playtestStatistics.loc[groupLabel, respondentsTwiceCountLabel]\
                                                - playtestStatistics.loc[groupLabel, respondentsExploitableCountLabel]
minus[respondentsTwiceVolunteersCountLabel] = playtestStatistics.loc[groupLabel, respondentsTwiceVolunteersCountLabel]\
                                                - playtestStatistics.loc[groupLabel, respondentsTwiceCountLabel]

In [ ]:
if displayResults:
#if True: 
    # source: https://flothesof.github.io/sankey-tutorial-matplotlib.html
    fig = plt.figure(figsize=(12, 12))
    ax = fig.add_subplot(1, 1, 1, xticks=[], yticks=[])
    diagrams = Sankey(flows=[
               #input
                        players,
               #output
                        -players-minus.sum(),
               #losses
                        minus[respondentsTwiceVolunteersCountLabel], 
                        minus[respondentsTwiceCountLabel], 
                        minus[respondentsExploitableCountLabel], 
                        minus[respondentsCountLabel] 
                ],

# English
           labels=[
               #input
                       playersCountLabel,
               #output
                       "exploitable sample",
               #losses
                       "non-respondents to optional questions",
                       "non-"+respondentsTwiceCountLabel,
                       "technical error",
                       "non-"+respondentsCountLabel
                ],

## French
#           labels=[
#               #input
#                       "joueurs",
#               #output
#                       "échantillon exploitable",
#               #losses
#                       "n'ont pas répondu\naux questions facultatives",
#                       "n'ont pas répondu une fois\nau prétest et au posttest",
#                       "problème technique",
#                       "n'ont répondu\nà aucun questionnaire"
#                ],

           orientations=[
               #input
                        0,
               #output
                        0,
               #losses
                        -1, -1, -1, -1
               ],
           rotation=-90,
            ax=ax,
           scale = 1/players
          ).finish()
    plt.title("Sankey diagram of experiment participants")
#    plt.title("Diagramme de Sankey des participants à l'expérience")
    for diagram in diagrams:
        diagram.text.set_fontweight('bold')
        diagram.text.set_fontsize('20')
        for text in diagram.texts:
            text.set_fontsize('14')

#### drafts

In [ ]:
gform[gform[QTemporality] == answerTemporalities[0]][QEnjoyed].apply(str).value_counts()

In [ ]:
gform[QAge].apply(str).value_counts()['nan']

In [ ]:
gform[gform[QAge].apply(str) == 'nan'][QTemporality].unique()